In [69]:
import asyncio
from poke_env.player.player import Player
from poke_env.player.random_player import RandomPlayer
import time
import pandas as pd
from BattleNode import BattleNode
from maxDamage import MaxDamagePlayer

In [70]:
# Dictionary to map the enumerated integer value of a type to the written word of the type in uppercase
# not currently using anymore but keeping just in case
pokeTypeDict = {
	1 : "BUG",
	2 : "DARK",
	3 : "DRAGON",
	4 : "ELECTRIC",
	5 : "FAIRY",
	6 : "FIGHTING",
	7 : "FIRE",
	8 : "FLYING",
	9 : "GHOST",
	10 : "GRASS",
	11 : "GROUND",
	12 : "ICE",
	13 : "NORMAL",
	14 : "POISON",
	15 : "PSYCHIC",
	16 : "ROCK",
	17 : "STEEL",
	18 : "WATER"
}

In [71]:
typeChart = pd.read_csv('TypeTableEdited.csv')
typeChart.rename(columns = {'Unnamed: 0' : 'AttackType'}, inplace = True)
typeChart = typeChart.set_index('AttackType')

# each row is an attacking type
# each column is the defending type

# print(typeChart) #uncomment to see table

In [72]:
def calcDamage(node):
	if node.player_turn:
		
        # Prepare some variables
		player_t1 = node.battle.active_pokemon.type_1
		player_t2 = ""
		player_m_t = node.move.type
		move_basepow = node.move.base_power
		opp_t1 = node.battle.opponent_active_pokemon.type_1
		opp_t2 = ""
		
		# Check if our pokemon has 2nd type
		if node.battle.active_pokemon.type_2 != None:
			player_t2 = node.battle.active_pokemon.type_2
		
		# Check if Opponent pokemon has 2nd type
		if node.battle.opponent_active_pokemon.type_2 != None:
			opp_t2 = node.battle.opponent_active_pokemon.type_2
		
		# First: check for stab
		stab_bonus = 1 #Default value, means no bonus

		if player_t2 != "":
			if player_t1 == player_m_t or player_t2 == player_m_t: #if type1 == move type or type 2 == move type, then stab bonus
				stab_bonus =  1.5
		else:
			if player_t1 == player_m_t: #if type1 == move type, then stab bonus
				stab_bonus =  1.5
		
		# Second: Get multiplier
		# Checks the DataFrame in pokeType for the multiplier for this move
		# get multi for type 1
		t1_multi = typeChart.loc[player_m_t.name, opp_t1.name]
		t2_multi = ""
		if opp_t2 != "" : #two types for opponent
			t2_multi = typeChart.loc[player_m_t.name, opp_t2.name]

		# Third: multiply stab, multis, and base power
		if t2_multi != "" :
			return  stab_bonus * t1_multi * t2_multi * move_basepow
		else:
			return stab_bonus * t1_multi * move_basepow


	else: #is opponent turn
		player_t1 = node.battle.active_pokemon.type_1
		player_t2 = ""
		opp_mt = node.move.type
		move_basepow = node.move.base_power
		opp_t1 = node.battle.opponent_active_pokemon.type_1
		opp_t2 = ""
		
		# Check if our pokemon has 2nd type
		if node.battle.active_pokemon.type_2 != None:
			player_t2 = node.battle.active_pokemon.type_2
		
		# Check if Opponent pokemon has 2nd type
		if node.battle.opponent_active_pokemon.type_2 != None:
			opp_t2 = node.battle.opponent_active_pokemon.type_2
		
		# First: check for stab
		stab_bonus = 1 #Default value, means no bonus

		if opp_t2 != "":
			if opp_t1 == opp_mt or opp_t2 == opp_mt: #if type1 == move type or type 2 == move type, then stab bonus
				stab_bonus =  1.5
		else:
			if opp_t1 == opp_mt: #if type1 == move type, then stab bonus
				stab_bonus =  1.5
		
		# Second: Get multiplier
		# Checks the DataFrame in pokeType for the multiplier for this move
		# get multi for type 1
		t1_multi = typeChart.loc[opp_mt.name, player_t1.name]
		t2_multi = ""
		if player_t2 != "" : #two types for opponent
			t2_multi = typeChart.loc[opp_mt.name, player_t2.name]

		# Third: multiply stab, multis, and base power
		if t2_multi != "" :
			return  stab_bonus * t1_multi * t2_multi * move_basepow
		else:
			return stab_bonus * t1_multi * move_basepow

In [73]:
def buildTree(node):
	'''	Builds a 2 or 3 level tree of possible moves for the current battle state.
		Returns root node which is the current battle state. '''
	if(node.level != None and node.level >= 3):
		if node.player_turn == False:
			raise Exception("Base node is opponent turn. Base nodes should only be for the player's turn.")
		return node #only want 3 layers, so stop making children
	else:
		if(node.player_turn == None or node.player_turn == False): #if first layer (none) or if parent is opponent turn, now it is player turn
			# Create array where status moves are at the end
			moves = []
			statMoves = []
			for m in node.battle.available_moves:
				if m.base_power != 0:
					moves.append(m)
				else:
					statMoves.append(m)
			moves = moves + statMoves

			# Add each available move as a node for the player's turn
			for i in range(0, len(moves)):
				child = BattleNode(node.battle, True, node)
				if node.level == None:
						child.level = 1
				else: child.level = node.level + 1
				child.move = (moves[i])

				# Calculate the possible damage for that move
				child.damage = calcDamage(child)

				# Get the state representation of the node
				estState(child)

				# go to next layer of branch
				buildTree(child)
				# when it comes back, add to parent
				node.addChild(child)

			# Add node(s) for switch ? or only switch on faint
				# make child
				# call buildTree on child
				# add child to parent
		else:
			# If it is the opponent's turn, then nodes are a few of their possible moves
			if len(node.battle.opponent_active_pokemon.moves) < 2:
				# If we do not know any possible moves yet, or only 1, just return to the parent and don't make another layer
				return
			else:
				# create a node for each of the known moves
				for move in node.battle.opponent_active_pokemon.moves.values(): 
					child = BattleNode(node.battle, False, node)
					if node.level == None:
						child.level = 1
					else: child.level = node.level + 1
					child.move = move 

					# Calculate the possible damage for that move
					child.damage = calcDamage(child)

					# Get the state representation of the node
					estState(child)
					
					# go to next layer of branch
					buildTree(child)
					
					# when it comes back, add to parent
					node.addChild(child)

				# Add node(s) for switch ? or only switch on faint
					# make child
					# call buildTree on child
					# add child to parent

		# return node, should be the root when all the recursive calls come back
		return node

In [74]:
def showNodeTree(root):
	print("Root: ", root.id, root.player_turn)
	i = 0
	for child in root.children:
		print("\tPlayer Node ", i, " : ", child)
		if len(child.children) > 0:
			j = 0
			for c in child.children:
				print("\t\t Opp Node ", j, " : ", c)
				if len(c.children) > 0:
					k = 0
					for kid in c.children:
						print("\t\t\tplayer node ", k, " : ", kid)
						k+=1
				j+=1
		i+=1

In [75]:

def estState(node):
    # Take possible damage from each parent, and estimate the state of our hp and the opponent hp
    # Go from child to parent and then back down, changing the values on the way down
    # Assign the value to the node as the game state value which is a dictonary of
        # est player hp: value
        # est opp hp: value
    damageDivisor = 100.0


    if node.level == 1: #if it is the first level, then get the hp values from the source
        opp_hp_fraction = node.battle.opponent_active_pokemon.current_hp_fraction
        player_damage_fraction = node.damage / damageDivisor

        player_hp = node.battle.active_pokemon.current_hp
        opponent_damage = node.damage

    elif node.level > 1: #if it is after the first level, then use the previous node's state
        opp_hp_fraction = node.parent.state.get("OppHP")
        player_damage_fraction = node.damage / damageDivisor

        player_hp = node.parent.state.get("PlayerHP")
        opponent_damage = node.damage

    playerTotal = node.battle.active_pokemon.current_hp
    oppHpOrig = node.battle.opponent_active_pokemon.current_hp_fraction

    if node.player_turn:
        # Contains player move: calc affect on opponent HP
        node.state = {
            "PlayerHP"  :  player_hp,
            "OppHP"     :  opp_hp_fraction - player_damage_fraction,
            "PlayerTot" :  playerTotal, 
            "OppOrig"   :  oppHpOrig
        }
    
    else: 
        # Contains opponent move: calc affect on player HP
        node.state = {
            "PlayerHP"  :  player_hp - opponent_damage,
            "OppHP"     :  opp_hp_fraction,
            "PlayerTot" :  playerTotal, 
            "OppOrig"   :  oppHpOrig
        }
    

In [95]:
def minmax(node):
	'''	Goes through the battle tree recursively to evaluate the nodes.
		Then chooses a best move and sends that node back to choose_move.'''

	if len(node.children) < 1:
		# This is the last node of a branch, needs a value assigned
		# Based on the state, decide the value:
			# high value on opponent HP dropping and our Hp remaning
			# value lowers as opponent HP is higher, and if our HP lowers

		playerHP = node.state.get("PlayerHP")
		oppHP = node.state.get("OppHP")
		playerTotal = node.battle.active_pokemon.current_hp
		oppHpOrig = node.battle.opponent_active_pokemon.current_hp_fraction

		# Calc value for oppHP

		if oppHP < -.60: 
			node.value = 20

		elif oppHP >= -.60 and oppHP < -.40 :
			node.value = 18

		elif oppHP >= -.40 and oppHP < -.30 :
			node.value = 16

		elif oppHP >= -.30 and oppHP < -.20 :
			node.value = 14

		elif oppHP >= -.20 and oppHP < -.10 :
			node.value = 12

		elif oppHP >= -.10 and oppHP < 0 :
			node.value = 10

		elif oppHP >= 0 and oppHP < .10 :
			node.value = 8

		elif oppHP >= .10 and oppHP < .20 :
			node.value = 6

		elif oppHP >= .20 and oppHP < .30 :
			node.value = 4

		elif oppHP >= .30 and oppHP < .40 :
			node.value = 2

		elif oppHP >= .40 and oppHP < .50 :
			node.value = 0

		elif oppHP >= .50 and oppHP < .60 :
			node.value = -2

		elif oppHP >= .60 and oppHP < .70 :
			node.value = -4

		elif oppHP >= .70 and oppHP < .80 :
			node.value = -6

		elif oppHP >= .80:
			node.value = -8
		
		else: #should not happen
			print("1 I didn't get a value so I am set to 0: PHP: ", playerHP, " PhpTotal: ", playerTotal, " OppHP: ", oppHP, " OppHpOrig: ", oppHpOrig)
			print( "\t oppHP < -.10", oppHP < -.10, " oppHP < -.10", oppHP < -.50, " oppHP < -.10", oppHP < -.10)
			node.value = 0
			
		# Calc value for PlayerHP
		if playerHP >= .75*playerTotal:
			node.value += 5

		elif (playerHP > .25*playerTotal and playerHP < .80*playerTotal):  
			node.value += 0

		elif playerHP <= .25*playerTotal:
			node.value += -5

		else: #should not happen
			print("2 I didn't get a value so I am set to 0: PHP: ", playerHP, " PhpTotal: ", playerTotal, " OppHP: ", oppHP, " OppHpOrig: ", oppHpOrig)
			node.value = 0
		
		return
			
	else:

		# go to end of each branch recursively
		for i in range(0, len(node.children)):
			minmax(node.children[i])

		# Comes back when all children have a value
		# Get value for node from children based on turn
		if node.player_turn: 
			minVal = 9999 #picks min of opponent moves
			for child in node.children:
				if child.value < minVal:
					minVal = child.value
			node.value = minVal 
			
		else: #if opponent turn
			maxVal = -9999 #picks max of player moves
			for child in node.children:
				if child.value > maxVal:
					maxVal = child.value
			node.value = maxVal 
	
	# if the node is the root (no parent)
	if node.parent == None:
		
		# find best child and return their move
		if node.player_turn == None: # root should always be player turn
			# Pick the highest of player moves for chosen move for root
			best_move = None
			maxVal = -9999
			for child in node.children:
				if child.value > maxVal:
					maxVal = child.value
					best_move = child.move

			# print("\nChosen move and value: ", best_move, maxVal)
			return best_move

		else: 
			print("Error: root not player turn")
			return 

In [97]:
class MinMaxPlayer(Player):
	
	def choose_move(self, battle):
		if battle.available_moves:
			
			rootNode = BattleNode(battle, None, None) #Create root node to start

			rootNode = buildTree(rootNode) #send root, returns root node of constructed tree
			
			best_move = minmax(rootNode) # send in root node(current battle), get chosen move
			
			if battle.can_dynamax: #if dynamax is available, use it
				return self.create_order(best_move, dynamax=True)

			# showNodeTree(rootNode) #Uncomment this to get info about the tree for each choice
			# if len(rootNode.children[0].children) > 1:
			# 	showNodeTree(rootNode)
			# print(battle.active_pokemon)
			# for c in rootNode.children: 
			# 	print(c)
			# if rootNode.children[0].move != best_move:
			# 	print("*********did not pick first move*******")
			# 	if len(rootNode.children[0].children) != 0:
			# 		showNodeTree(rootNode)
			# 	else: 
			# 		print("only one level")
				
			return self.create_order(best_move)
		else: 
			return self.choose_random_move(battle)

In [98]:

#Create random Opponent
random_player = RandomPlayer(battle_format="gen8randombattle")

#Create maxDamage opponent
maxDamage_player = MaxDamagePlayer(battle_format="gen8randombattle")

#Create minMaxPlayer
mmP = MinMaxPlayer(battle_format="gen8randombattle")

numBattles = 50

start = time.time()
# run for set number of games
await mmP.battle_against(random_player, n_battles=numBattles)

# print results 
print(
	"MinMax player won %d / %d battles against random player [this took %f seconds]"
	% (
		mmP.n_won_battles, numBattles, time.time() - start
	)
)

# print("\n\n\n\n\n")

mmP2 = MinMaxPlayer(battle_format="gen8randombattle")
start = time.time()
await mmP2.battle_against(maxDamage_player, n_battles=numBattles)
# print results 
print(
	"MinMax player won %d / %d battles against maxDamage player [this took %f seconds]"
	% (
		mmP2.n_won_battles, numBattles, time.time() - start
	)
)

MinMax player won 47 / 50 battles against random player [this took 9.538029 seconds]
MinMax player won 44 / 50 battles against maxDamage player [this took 7.098370 seconds]


In [79]:
# Things we could try to improve it

# Implement smart switch after faint. Trouble is i can't figure out when showdown makes that choice and if I can change it.
# So for right now, i am working on adding the option to switch when we are at a big disadvantage after the opponent has used at least one move
# Could also consider switching only based on their poke type vs ours, but that would prevent us from using a move that we have that is supereffective if it is not the same as our types. 

# Use dynamax smarter? like if the value for a node is less than some value, then check if we can use dynamax
    # currently it is used in choose_move and usually uses it on first turn

# Check on lvl 3 if any of the moves only had one pp left, and do not make a node for it if it's grandparent (node.parent.parent) used that move

# Consider trying to get the list of possible moves for a pokemon from this https://pkmn.github.io/randbats/ 

# Other ideas?

# Note, uncomment line 12 in class MinMaxPlayer to get info about the tree and see the values 
# Can also change the number of battles to run in the above cell. I recomend only running 1 and commenting out one of the battles. 
